In [111]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

import tensorflow as tf
import keras
from keras import layers

In [112]:
df = pd.read_csv("housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [113]:
print("nan:\n", df.isna().sum())

nan:
 longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64


In [114]:
df = df.drop('total_bedrooms', axis=1)
print("nan:\n", df.isna().sum())

nan:
 longitude             0
latitude              0
housing_median_age    0
total_rooms           0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64


In [115]:
df['ocean_proximity'] = LabelEncoder().fit_transform(df['ocean_proximity'])

# CatBoost requires we save our continuous and categorical variables separately into a list
categorical_variables = ['ocean_proximity']

# continous variables also into a list
continuous_variables = ['longitude', 'latitude', 'housing_median_age', 'total_rooms	', 'total_bedrooms', 'population', 'households', 'median_income']

# X/y 
X = df.drop("median_house_value", axis=1)
y = df['median_house_value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# SCALING => some of the algorithms require this
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [116]:
models = {
    'Linear Regression': LinearRegression(),
    'SVM': SVR(),
    'KNN': KNeighborsRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': xgb.XGBRegressor(enable_categorical=True, objective='reg:squarederror'),
    'CatBoost': cb.CatBoostRegressor(verbose=0),
    'LightGBM-default': lgb.LGBMRegressor(objective='regression'),
    'LightGBM-optimized': lgb.LGBMRegressor(objective='regression', learning_rate=0.1638181818181818, max_depth=9, num_leaves=51)
    
}

In [117]:
# print("Nan в X_train:", X_train.isna().sum().sum())
# print("Nan в X_test:", X_test.isna().sum().sum())

In [118]:
# our benchmarking code!

# this will contain later our results of the benchmark
results = []

# loop through our models above
for name, model in models.items():

    # we have to react to certain algorithms
    # because they have requirements for fit()

    print("Starting ... " + name)

    # SVM / KNN require scaled data
    if name in ['SVM', 'KNN']:
        model.fit(X_train_scaled, y_train)
        predictions = model.predict(X_test_scaled)
    elif name == 'CatBoost':
        # CatBoost requires
        # we specify which columns are categories, INCLUDING ORDINALS
        model.fit(X_train, y_train, cat_features=categorical_variables)
        predictions = model.predict(X_test)
    else:
        # everything else follows the same logic!
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

    # Calculate metrics for current model in training
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)

    # save the metrics for this model into results
    results.append([name, mae, mse, rmse, r2])

Starting ... Linear Regression
Starting ... SVM
Starting ... KNN
Starting ... Random Forest
Starting ... XGBoost
Starting ... CatBoost
Starting ... LightGBM-default
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000391 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1588
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 207336.643047
Starting ... LightGBM-optimized
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000317 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1588
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 207336.643047
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

In [119]:
metrics_df = pd.DataFrame(results, columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2'])

light_theme = 'background: yellow'
dark_theme = 'background: goldenrod'

# you can alternate the highlight styling based on your theme
default_highlight_style = dark_theme

# helper function that highlights the best model of each metric
def highlight_best_metrics(row):
    # default styles for everything is empty in the beginning
    styles = ['' for _ in row]

    # index 1 => MAE (index 0 => model name)
    if row['MAE'] == metrics_df['MAE'].min():
        styles[1] = default_highlight_style

    # index 2 = > MSE
    if row['MSE'] == metrics_df['MSE'].min():
        styles[2] = default_highlight_style

    # index 3 = > RMSE
    if row['RMSE'] == metrics_df['RMSE'].min():
        styles[3] = default_highlight_style

    # index 4 = > R-squared (R2)
    if row['R2'] == metrics_df['R2'].max():
        styles[4] = default_highlight_style

    return styles


# apply the custom styles based on the min/max metrics
highlight_df = metrics_df.style.apply(highlight_best_metrics, axis=1)


highlight_df

,Model,MAE,MSE,RMSE,R2
0,Linear Regression,51759.698586,4975881250.551231,70539.926641,0.627870
1,SVM,87342.158235,13883557107.357630,117828.507193,-0.038307
2,KNN,40546.382946,3736422844.571764,61126.286036,0.720565
3,Random Forest,31850.245673,2445032598.561617,49447.270891,0.817144
4,XGBoost,31282.825767,2271329984.835232,47658.472330,0.830135
5,CatBoost,30161.532860,2137746107.783558,46235.766543,0.840125
6,LightGBM-default,31826.995621,2287689050.264125,47829.792497,0.828911
7,LightGBM-optimized,30406.204239,2144873214.774440,46312.775935,0.839592


In [ ]:
# MAE: 30866.4550603738
# MSE: 2271129136.945447
# RMSE: 47656.36512519023
# R2: 0.8340874655368169
#not usre I did everything right, I dropped a columns because even if Ireplace nan with mean beofre training after traning there would be nan values 
#which would make impossible to benchmarking 